In [1]:
import pandas as pd
from pathlib import Path

# Ścieżka do pliku JSON
json_file = Path("../data/all_offers.json")


# Wczytanie danych do DataFrame
df = pd.read_json(json_file)

# Usunięcie kolumny 'zdjecia'
df = df.drop(columns=["zdjecia"], errors="ignore")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14679 entries, 0 to 14678
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 14679 non-null  int64  
 1   marka              14679 non-null  object 
 2   model              14679 non-null  object 
 3   rok_produkcji      14679 non-null  int64  
 4   przebieg           14678 non-null  object 
 5   rodzaj_paliwa      14679 non-null  object 
 6   pojemnosc_silnika  14246 non-null  object 
 7   moc_silnika        14665 non-null  object 
 8   skrzynia_biegow    14676 non-null  object 
 9   naped              13729 non-null  object 
 10  liczba_drzwi       14629 non-null  float64
 11  typ_nadwozia       14679 non-null  object 
 12  kolor              14679 non-null  object 
 13  liczba_miejsc      13253 non-null  float64
 14  stan_pojazdu       14679 non-null  object 
 15  bezwypadkowosc     14679 non-null  bool   
 16  cena               146

In [3]:
df['czy_cesja'].sum()

np.int64(3007)

In [4]:
df['czy_uszkodzony'].sum()

np.int64(718)

In [5]:
df[(df['czy_cesja']) | (df['czy_uszkodzony'])].shape[0]

3720

In [6]:
df_cleaned = df[~((df['czy_cesja']) | (df['czy_uszkodzony']))]

In [7]:
df_cleaned['liczba_miejsc'] = df_cleaned['liczba_miejsc'].fillna(5)
df_cleaned['naped'] = df_cleaned['naped'].fillna("Na tylne koła")

C:\Users\tthaddey\AppData\Local\Temp\ipykernel_16824\3364323884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['liczba_miejsc'] = df_cleaned['liczba_miejsc'].fillna(5)
C:\Users\tthaddey\AppData\Local\Temp\ipykernel_16824\3364323884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['naped'] = df_cleaned['naped'].fillna("Na tylne koła")


In [8]:
df_cleaned = df_cleaned.dropna()

In [9]:
from currency_converter import CurrencyConverter
c = CurrencyConverter()

def convert_to_pln(row):
    if row['waluta'] != 'PLN':  # Sprawdź, czy waluta jest różna od PLN
        try:
            # Konwersja ceny do PLN
            return c.convert(row['cena'], row['waluta'], 'PLN')
        except ValueError as e:
            print(f"Błąd przy konwersji dla ID {row['id']}: {e}")
            return None  # Jeśli nie uda się przekonwertować
    return row['cena']  # Jeśli waluta to PLN, zwróć oryginalną cenę

df_cleaned['cena'] = df_cleaned.apply(convert_to_pln, axis=1)
df_cleaned['waluta'] = 'PLN'


In [12]:
df_cleaned = df_cleaned.loc[:, df_cleaned.nunique() > 1]

In [15]:
# Konwersja kolumn na wartości numeryczne
df_cleaned['przebieg'] = df_cleaned['przebieg'].str.replace(' km', '', regex=False).str.replace(' ', '').astype(float)
df_cleaned['pojemnosc_silnika'] = df_cleaned['pojemnosc_silnika'].str.replace(' cm3', '', regex=False).str.replace(' ', '').astype(float)
df_cleaned['moc_silnika'] = df_cleaned['moc_silnika'].str.replace(' KM', '', regex=False).str.replace(' ', '').astype(float)

In [16]:
df_cleaned.nunique()

id                   10714
model                   37
rok_produkcji           52
przebieg              4003
rodzaj_paliwa            5
pojemnosc_silnika       82
moc_silnika            166
skrzynia_biegow          2
naped                    5
liczba_drzwi             4
typ_nadwozia             9
kolor                   17
liczba_miejsc            7
stan_pojazdu             2
bezwypadkowosc           2
cena                  2285
czy_prywatny             2
lokalizacja           1832
url                  10714
dtype: int64

In [17]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10714 entries, 0 to 14677
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10714 non-null  int64  
 1   model              10714 non-null  object 
 2   rok_produkcji      10714 non-null  int64  
 3   przebieg           10714 non-null  float64
 4   rodzaj_paliwa      10714 non-null  object 
 5   pojemnosc_silnika  10714 non-null  float64
 6   moc_silnika        10714 non-null  float64
 7   skrzynia_biegow    10714 non-null  object 
 8   naped              10714 non-null  object 
 9   liczba_drzwi       10714 non-null  float64
 10  typ_nadwozia       10714 non-null  object 
 11  kolor              10714 non-null  object 
 12  liczba_miejsc      10714 non-null  float64
 13  stan_pojazdu       10714 non-null  object 
 14  bezwypadkowosc     10714 non-null  bool   
 15  cena               10714 non-null  float64
 16  czy_prywatny       10714 no

In [ ]:
df_cleaned.head(10)

In [18]:
from pathlib import Path
# Ścieżka do pliku wyjściowego
output_file = Path("../data/clean_data.json")
# Zapis DataFrame do pliku JSON
df_cleaned.to_json(output_file, orient='records', lines=True, force_ascii=False)